In [6]:
import re
import json
from timeout_decorator import timeout


In [ ]:
file_name = "RegexEval_Filtered_Manual"

In [7]:
with open(f"./Intermediate_Dataset/{file_name}.json") as f:
    data = json.load(f)

len(data)

1505

In [8]:
@timeout(60)
def match_expression(expression, text):
    try:
        return re.search(expression, text) is not None
    except Exception as e:
        print(e)
        return None

In [9]:
new_data = []
for item in data:
    if item['expression'].strip() == '' or item['description'].strip() == '' or item['matches'].strip() == '' or item['non_matches'].strip() == '':
        continue
    
    is_valid = True
    expression = item['expression'].strip()
    description = item['description'].strip()
    matches = [x.strip() for x in item['matches'].strip().split('|')]
    non_matches = [x.strip() for x in item['non_matches'].strip().split('|')]


    for match in matches:
        try:
            is_match = match_expression(expression, match)
        except TimeoutError:
            print('TimeoutError: ', expression, match)
            is_match = None
        if is_match is None or is_match == False:
            is_valid = False
            break    
    for non_match in non_matches:
        try:
            is_match = match_expression(expression, non_match)
        except TimeoutError:
            print('TimeoutError: ', expression, match)
            is_match = None
        if is_match is None or is_match == True:
            is_valid = False
            break

    if is_valid:
        new_data.append(item)
    else:
        print('Invalid: ', expression, description, matches, non_matches)


Invalid:  ^-?([1]?[1-7][1-9]|[1]?[1-8][0]|[1-9]?[0-9])\.{1}\d{1,6} Longitude validation (similiar to Latitude Validation) and inspired by the same code ['180.0', '-180.0', '98.092391'] ['181', '180', '-98.0923913']


/var/folders/9d/98myd0lx1rn2j76hx3t96x1r0000gn/T/ipykernel_12898/1341296894.py:4: FutureWarning: Possible nested set at position 5
  return re.search(expression, text) is not None
/var/folders/9d/98myd0lx1rn2j76hx3t96x1r0000gn/T/ipykernel_12898/1341296894.py:4: FutureWarning: Possible nested set at position 21
  return re.search(expression, text) is not None
/var/folders/9d/98myd0lx1rn2j76hx3t96x1r0000gn/T/ipykernel_12898/1341296894.py:4: FutureWarning: Possible nested set at position 27
  return re.search(expression, text) is not None


In [10]:
with open(f"./Intermediate_Dataset/{file_name}_Filtered.json", 'w') as f:
    json.dump(new_data, f)